Huggingface需要手动将模型和参数放到GPU上
```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
sentence  = 'Hello World!'
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
model     = BertModel.from_pretrained('bert-large-uncased')

inputs    = tokenizer(sentence, return_tensors="pt").to(device)
model     = model.to(device)
outputs   = model(**inputs)
```

In [1]:
import ffmpeg

def extractAudio(input_file, output_file):
    stream = ffmpeg.input(input_file)
    audio = stream.audio
    audio = ffmpeg.output(audio, output_file)
    ffmpeg.run(audio)

In [2]:
extractAudio("./data/bda-030.mp4","./data/bda-030.mp3")

In [1]:
from pydub import AudioSegment
import torchaudio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq




In [ ]:

# 读取音频文件
audio_file = AudioSegment.from_file("./data/bda-030.mp3", format="mp3")

# 指定开始和结束时间
start_time = 1 * 60 * 1000  # 1分钟
end_time = 2 * 60 * 1000  # 2分钟

# 提取部分片段
audio_segment = audio_file[start_time:end_time]

# 将采样率设置为目标采样率
target_sample_rate = 16000
audio_segment = audio_segment.set_frame_rate(target_sample_rate)

# 将片段转换为wav格式并保存到内存中
audio_segment.export("temp.wav", format="wav")

In [2]:
# 使用torchaudio读取wav文件
waveform, sample_rate = torchaudio.load("temp.wav")

# 加载Wav2Vec2模型和处理器
model = AutoModelForSpeechSeq2Seq.from_pretrained("vumichien/whisper-large-v2-jp")
processor = AutoProcessor.from_pretrained("vumichien/whisper-large-v2-jp")

In [8]:
waveform

tensor([[ 2.1973e-03,  2.2278e-03,  8.5449e-04,  ..., -1.2512e-03,
         -3.0518e-05,  1.3123e-03],
        [ 2.0142e-03,  2.3804e-03,  1.4954e-03,  ..., -2.7466e-04,
         -2.1362e-04,  7.3242e-04]])

In [3]:
from datasets import load_dataset

In [4]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

Found cached dataset librispeech_asr_dummy (C:/Users/wangz/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [5]:
ds[0]["audio"]["array"].shape

(93680,)

In [11]:
import torch

In [6]:
waveform[0].numpy().shape

(960000,)

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [13]:
# 通过处理器处理音频
inputs = processor(waveform[0].numpy(), sampling_rate=sample_rate, return_tensors="pt").to(device)

model.to(device)

# 获取输入特征
input_features = inputs.input_features

# 生成ID
generated_ids = model.generate(input_features)

# 转录
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(transcription)

d:\anaconda3\envs\huggingface\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


お 仕事 , 大変 だっ た もの ? いつ も あなた , 疲れ て 寝 ちゃう の を . 寝 , たまに も いい です . 腹 . 

